#### 우주 함선에서 살아남을 확률
 - 데이터셋 : spaceship
 - feature : 
 - target : Transported
 - 알고리즘 :  

[1] 모듈 로딩 및 데이터 준비

In [153]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [154]:
spaceDF = pd.read_csv(r'C:\Users\KDP-17\EX_PANDAS6\MachineLearning\data\spaceship\train.csv')
testDF = pd.read_csv(r'C:\Users\KDP-17\EX_PANDAS6\MachineLearning\data\spaceship\test.csv')

In [155]:
# 형준님 전처리 된 거 가져오기
# 모든 금액이 결측치인 데이터 체크 없으므로 개인별 총 지불한 금액 평균치를 대입하기로 결정
space_cols =spaceDF.columns[[7,8,9,10,11]].to_list()
spaceDF[space_cols].isna().all(axis=1).sum()


spaceDF.drop(index=spaceDF[spaceDF['Cabin'].isna()].index,inplace=True)
spaceDF.loc[spaceDF['Age'].isna(),'Age'] = spaceDF['Age'].mean()

vipCut = spaceDF.loc[spaceDF['VIP'] == True,space_cols].mean(axis=1).mean()

vipidx = spaceDF.loc[spaceDF['VIP'].isna(),'PassengerId']
# VIP 멤버들이 평균적으로 사용한 금액이상으로 썻다면 VIP로 대입
for idx in vipidx:
    h_m = spaceDF.loc[spaceDF['PassengerId'] == idx,space_cols].mean(axis=1).values[0]
    if h_m >= vipCut:
        spaceDF.loc[spaceDF['PassengerId'] == idx,'VIP'] = 'True'
    else:
        spaceDF.loc[spaceDF['PassengerId'] == idx,'VIP'] = 'False'

# 안쓰는 컬럼 삭제
spaceDF.drop(columns='PassengerId',inplace=True)
spaceDF.drop(columns='Name',inplace=True)

[1-1] 데이터 정제(인코딩)

In [156]:
# object 타입 데이터 원-핫 인코딩
ex_featureDF = spaceDF[spaceDF.columns[:-1]]
targetSR = spaceDF[spaceDF.columns[-1]]

ex_featureDF

floatDF = ex_featureDF[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]
objDF = ex_featureDF[['HomePlanet','CryoSleep','Destination','VIP']]

In [157]:
# 원-핫인코딩 - feature
objDF_ohe = pd.get_dummies(objDF, columns=objDF.columns,drop_first=True)
objDF_ohe = objDF_ohe.applymap(lambda x: 1 if x is True else (0 if x is False else x))
objDF_ohe

,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,VIP_False,VIP_True
0,1,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,1,0,0,0,1,1,0,0
3,1,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
8688,1,0,0,0,0,1,0,0
8689,0,0,1,1,0,0,0,0
8690,0,0,0,0,1,0,0,0
8691,1,0,0,0,0,0,0,0


In [158]:
print(objDF_ohe.shape, floatDF.shape)
objDF_ohe.astype('float64')

(8494, 8) (8494, 6)


,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,VIP_False,VIP_True
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
8688,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8691,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
# 인코딩 완
featureDF = pd.concat([floatDF,objDF_ohe],axis=1)
featureDF

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,0,0,0,0,1,0,0,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0,0,0,1,1,0,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,1,0,0,0,1,0,0,0
4,16.0,303.0,70.0,151.0,565.0,2.0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,1,0,0,0,0,1,0,0
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0,0,0
8690,26.0,0.0,0.0,1872.0,1.0,0.0,0,0,0,0,1,0,0,0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,1,0,0,0,0,0,0,0
